# Sampling and Splitting

## Sampling

Sampling for basic tabular datasets. (Not designed for time series as of now.)

In [1]:
import polars as pl
import polars_ds as pds
import polars_ds.sample_and_split as sa

In [2]:
df = pds.frame(size=100_000).with_columns(
    pds.random(0.0, 12.0).alias("uniform_1"),
    pds.random(0.0, 1.0).alias("uniform_2"),
    pds.random_exp(0.5).alias("exp"),
    pds.random_normal(0.0, 1.0).alias("normal"),
    pds.random_normal(0.0, 1000.0).alias("fat_normal"),
    (pds.random_int(0, 3)).alias("flags"),
    pl.Series(["A"] * 30_000 + ["B"] * 30_000 + ["C"] * 40_000).alias("category"),
)
df.head()

row_num,uniform_1,uniform_2,exp,normal,fat_normal,flags,category
i64,f64,f64,f64,f64,f64,i32,str
0,3.169978,0.183096,0.848878,-0.988939,369.76195,2,"""A"""
1,8.810768,0.569672,0.048483,-0.44255,258.012662,0,"""A"""
2,3.274063,0.632772,0.447468,0.255512,-1284.389879,1,"""A"""
3,10.847672,0.89006,0.772062,0.735149,-0.362983,0,"""A"""
4,11.66482,0.907167,1.393929,2.285448,-2031.321622,0,"""A"""


In [3]:
sa.random_cols(df, 2, keep = ["row_num"])

['row_num', 'normal', 'flags']

In [4]:
# Random Sample
sa.sample(df, 0.6) # by ratio

row_num,uniform_1,uniform_2,exp,normal,fat_normal,flags,category
i64,f64,f64,f64,f64,f64,i32,str
1,8.810768,0.569672,0.048483,-0.44255,258.012662,0,"""A"""
2,3.274063,0.632772,0.447468,0.255512,-1284.389879,1,"""A"""
4,11.66482,0.907167,1.393929,2.285448,-2031.321622,0,"""A"""
6,1.522247,0.626331,0.460844,-0.060739,1487.444343,1,"""A"""
7,3.93548,0.363229,2.002222,-0.613627,-335.203183,0,"""A"""
…,…,…,…,…,…,…,…
99991,3.808594,0.693914,6.727779,-0.781093,-868.307031,2,"""C"""
99994,6.246362,0.99597,3.468162,-0.699768,-145.471814,1,"""C"""
99996,0.520435,0.758179,0.680518,0.788875,-3203.56896,2,"""C"""


In [5]:
sa.sample(df, 30_000) # by count

row_num,uniform_1,uniform_2,exp,normal,fat_normal,flags,category
i64,f64,f64,f64,f64,f64,i32,str
0,3.169978,0.183096,0.848878,-0.988939,369.76195,2,"""A"""
12,2.718784,0.236327,0.656341,2.042461,992.106646,0,"""A"""
13,5.688242,0.238128,1.989903,-1.890975,96.609098,1,"""A"""
16,10.630157,0.685417,2.040244,-0.411343,-80.440654,2,"""A"""
18,6.133318,0.868581,3.786928,-0.853489,-824.372864,1,"""A"""
…,…,…,…,…,…,…,…
99976,7.686265,0.037178,9.872401,0.002709,1013.823443,2,"""C"""
99981,9.040585,0.272563,0.423536,-0.365252,-718.151462,1,"""C"""
99985,8.940385,0.856215,2.355023,0.609717,-34.944096,0,"""C"""


In [6]:
df.group_by("flags").len().sort("flags")

flags,len
i32,u32
0,33465
1,33331
2,33204


In [7]:
# Downsample on one group
sa1 = sa.downsample(
    df, 
    (pl.col("flags") == 0, 0.5)
)
sa1.group_by("flags").len().sort("flags")

flags,len
i32,u32
0,16732
1,33331
2,33204


In [8]:
# Downsample on multiple groups
sa2 = sa.downsample(
    df, 
    (pl.col("flags") == 0, 0.5),
    (pl.col("flags") == 1, 0.3),
    (pl.col("flags") == 2, 0.4),
)
sa2.group_by("flags").len().sort("flags")

flags,len
i32,u32
0,16732
1,9999
2,13281


In [9]:
df.group_by("category").len().sort("category")

category,len
str,u32
"""A""",30000
"""B""",30000
"""C""",40000


In [10]:
# Volume neutral by each category, will take the greatest possible value so that we get neutral volume.
vn = sa.volume_neutral(
    df,
    by = pl.col("category"),
)
vn.group_by("category").len().sort("category")

category,len
str,u32
"""A""",30000
"""B""",30000
"""C""",30000


In [11]:
# Volume neutral (10_000) by each category
vn = sa.volume_neutral(
    df,
    by = pl.col("category"),
    target_volume = 10_000
)
vn.group_by("category").len().sort("category")

category,len
str,u32
"""A""",10000
"""B""",10000
"""C""",10000


In [12]:
# Volume neutral (10_000) by a more complicated condition
vn = sa.volume_neutral(
    df,
    by = pl.col("category") == "A",
    target_volume = 10_000
) # This makes sense because count for B + count for C = 10_000
vn.group_by("category").len().sort("category")

category,len
str,u32
"""A""",10000
"""B""",4285
"""C""",5715


In [13]:
# Volume neutral sample with a control level. Volume neutral happens under the category level, meaning
# the volume for each flag in each category is neutral.
vn = sa.volume_neutral(
    df,
    by = pl.col("flags"),
    control = pl.col("category")
) 
vn.group_by(["category", "flags"]).len().sort(["category", "flags"])

category,flags,len
str,i32,u32
"""A""",0,9960
"""A""",1,9960
"""A""",2,9960
"""B""",0,9962
"""B""",1,9962
"""B""",2,9962
"""C""",0,13223
"""C""",1,13223
"""C""",2,13223


In [14]:
# We may not meet the target volume for all categories.
vn = sa.volume_neutral(
    df,
    by = pl.col("flags"),
    control = pl.col("category"),
    target_volume= 10_000
) # 
vn.group_by(["category", "flags"]).len().sort(["category", "flags"])

category,flags,len
str,i32,u32
"""A""",0,9960
"""A""",1,9960
"""A""",2,9960
"""B""",0,9962
"""B""",1,9962
"""B""",2,9962
"""C""",0,10000
"""C""",1,10000
"""C""",2,10000


## Splitting

Split by ratios.

In [15]:
print(df.shape)
train, test = sa.split_by_ratio(
    df,
    split_ratio = 0.6
)
print(train.shape)
print(test.shape)


(100000, 8)
(60000, 8)
(40000, 8)


In [16]:
print(df.shape)
for frame in sa.split_by_ratio(df, split_ratio = [0.25, 0.4, 0.10, 0.25]):
    print(frame.shape)

(100000, 8)
(25000, 8)
(40000, 8)
(10000, 8)
(25000, 8)
